In [1]:
import pandas as pd
#Import SpaCy
import spacy
import regex
from spacy import displacy
from collections import Counter
import copy
from fuzzywuzzy import process
#target_nlp = spacy.load('de_core_news_sm')
IN = "input/dev.roen.df.short.tsv"
OUT = "output/masked-levenshtein-1/dev.roen.masked.tsv"
JSON = "output/masked-levenshtein-1/dev.roen.mapping.json"
entities = ["PER", "PERSON", "ORG", "ORGANIZATION", "PRODUCT", "WORK_OF_ART", "LOC"]


In [9]:
def same_enity_mask(input_csv, output_csv, output_json, entities, lang):
#Set up dict & dataframe
    d = {}
    df = input_csv
    if lang == "roen":
        source_nlp = spacy.load("ro_core_news_sm")
    else:
        source_nlp = spacy.load("en_core_web_sm")

    entity_count = 0
    for index, row in df.iterrows():
        line1, line2 = row.original, row.translation
        source = source_nlp(line1)
        source_adjust, target_adjust = 0, 0
        
#mask source:
        for X in source.ents:
            if X.label_ in entities:
                if X.text not in d:
                    d[X.text] = "NE{}".format(entity_count)
                start, end = X.start_char-source_adjust, X.end_char-source_adjust
                line1 = "".join([line1[:start],d[X.text],line1[end:]])
                source_adjust=len(X.text)-len(d[X.text])
                
  
#Find NE from source in target and mask:
                entity = "".join(["(?b)" + "(", X.text, ")", "{0<=s<=5,0<=d<=5}"])
                re = regex.search(entity, line2)
                if re:
                    d[X.text] = "NE{}".format(entity_count)
                    t_start, t_end = re.span()[0]-target_adjust, re.span()[1]-target_adjust
                    line2 = "".join([line2[0:re.span()[0]], d[X.text], line2[re.span()[1]:]])
                    target_adjust=(re.span()[1]-re.span()[0])-len(d[X.text])
                entity_count+=1


#Replace row with masked version:
        df.loc[index,"original"]=line1
        df.loc[index,"translation"]=line2
    return df

In [2]:
def mask(input_csv, output_csv, output_json, entities, lang):
    
#Set up dict & dataframe
    d = {}
    df = input_csv

    if lang == "roen":
        source_nlp = spacy.load("ro_core_news_sm")
    else:
        source_nlp = spacy.load("en_core_web_sm")

    entity_count = 0
    for index, row in df.iterrows():
        line1, line2 = row.original, row.translation
        source = source_nlp(line1)
        source_adjust, target_adjust = 0, 0
        
#mask source:
        for X in source.ents:
            if X.label_ in entities:
                if X.text not in d:
                    d[X.text] = "NE{}".format(entity_count)
                start, end = X.start_char-source_adjust, X.end_char-source_adjust
                line1 = "".join([line1[:start],d[X.text],line1[end:]])
                source_adjust=len(X.text)-len(d[X.text])
                entity_count+=1
  
#Find NE from source in target and mask:
                entity = "".join(["(?b)" + "(", X.text, ")", "{0<=s<=5,0<=d<=5}"])
                re = regex.search(entity, line2)
                if re:
                    d[X.text] = "NE{}".format(entity_count)
                    t_start, t_end = re.span()[0]-target_adjust, re.span()[1]-target_adjust
                    line2 = "".join([line2[0:re.span()[0]], d[X.text], line2[re.span()[1]:]])
                    target_adjust=(re.span()[1]-re.span()[0])-len(d[X.text])
                    entity_count+=1


#Replace row with masked version:
        df.loc[index,"original"]=line1
        df.loc[index,"translation"]=line2
#Export dict to json:
    return df 

# EN-DE
This is a notebook detaiolign the masking of the en de dataset at different levels

## dev data

In [3]:
en_de_dev_df =  pd.read_csv("../data/en-de/dev.ende.df.short.tsv",sep="\t")

result_dev = pd.read_csv("../masked-data/dev_en_de.tsv", sep="\t")

result_dev

,text_a,text_b,labels,predictions
0,"Simultaneously, the Legion took part to the pa...",Gleichzeitig nahm die Legion an der Befriedung...,-0.312186,-0.345279
1,He also begins an affair with Veronica Harring...,Er beginnt auch eine Affäre mit Veronica Harri...,-0.401581,-0.284105
2,The urban morphology of these two local waters...,Die urbane Morphologie dieser beiden lokalen W...,0.275414,-0.322763
3,Important finds included a bronze axe in Wellw...,Wichtige Funde waren eine Bronzeaxt in Wellwoo...,0.580925,-0.302298
4,"Previously, Englishmen had drunk mainly dark s...",Früher hatten Engländer vor allem dunkle Stout...,-1.895129,-0.315449
...,...,...,...,...
995,"The gang absconded with $2,000 cash in the sec...",Die Bande flüchtete mit $2.000 Bargeld in den ...,0.088897,-0.325243
996,The Irish settlers arrives from Northern Irela...,Die irischen Siedler kommen kurz nach den Loya...,0.614926,-0.263670
997,Volcanics include dacite breccia and small rem...,Zu den Vulkanen gehören Dacite Breccia und kle...,-0.484296,-0.364864
998,The Fort Concho Museum acquired the Schoolhous...,Das Fort Concho Museum erwarb 1946 das Schulha...,0.560427,-0.238750


In [4]:
result_dev["error"] = abs(result_dev["labels"]-result_dev["predictions"])
result_dev

,text_a,text_b,labels,predictions,error
0,"Simultaneously, the Legion took part to the pa...",Gleichzeitig nahm die Legion an der Befriedung...,-0.312186,-0.345279,0.033093
1,He also begins an affair with Veronica Harring...,Er beginnt auch eine Affäre mit Veronica Harri...,-0.401581,-0.284105,0.117476
2,The urban morphology of these two local waters...,Die urbane Morphologie dieser beiden lokalen W...,0.275414,-0.322763,0.598177
3,Important finds included a bronze axe in Wellw...,Wichtige Funde waren eine Bronzeaxt in Wellwoo...,0.580925,-0.302298,0.883223
4,"Previously, Englishmen had drunk mainly dark s...",Früher hatten Engländer vor allem dunkle Stout...,-1.895129,-0.315449,1.579680
...,...,...,...,...,...
995,"The gang absconded with $2,000 cash in the sec...",Die Bande flüchtete mit $2.000 Bargeld in den ...,0.088897,-0.325243,0.414140
996,The Irish settlers arrives from Northern Irela...,Die irischen Siedler kommen kurz nach den Loya...,0.614926,-0.263670,0.878595
997,Volcanics include dacite breccia and small rem...,Zu den Vulkanen gehören Dacite Breccia und kle...,-0.484296,-0.364864,0.119432
998,The Fort Concho Museum acquired the Schoolhous...,Das Fort Concho Museum erwarb 1946 das Schulha...,0.560427,-0.238750,0.799178


In [5]:
result_dev = result_dev.sort_values(by=["error"],ascending=False)
result_dev = result_dev.head(10)
result_dev.reset_index(inplace=True,drop=True)
result_dev

,text_a,text_b,labels,predictions,error
0,World Encyclopedia of Police Forces and Penal ...,World Encyclopedia of Police Forces and Straal...,-4.825809,-0.340117,4.485691
1,In The Goldenseal Book of the West Virginia Mi...,In The Goldenseal Book of the West Virginia Mi...,-3.760660,-0.349047,3.411613
2,"When the wrestler hesitated or declined, Sting...","Als der Wrestler zögerte oder ablehnte, nickte...",-3.730812,-0.373251,3.357561
3,The Cubs tied it in the third on a triple by B...,Die Cubs band es in der dritten auf einem Trip...,-3.760030,-0.449195,3.310835
4,Possibly Old South Arabian speakers dwelled fr...,Möglicherweise wohnten alte südarabische Sprec...,-3.616978,-0.311670,3.305308
5,"After the race, Turcotte explained that ""when ...","Nach dem Rennen erklärte Turcotte, dass ""wenn ...",-3.501024,-0.388106,3.112918
6,"Taft Avenue Flyover, Metro Manila Interchange ...","Taft Avenue Flyover, Metro Manila Interchange ...",-2.928729,-0.313932,2.614798
7,"Charley Bowser, a Pitt end, coached at his alm...","Charley Bowser, ein Pitt-Ende, trainierte an s...",-2.781564,-0.307047,2.474517
8,The fictional Ryker's houses both conventional...,Die fiktive Ryker Häuser sowohl konventionelle...,-2.804584,-0.395844,2.408740
9,Sienna grows feelings for Darren and they kiss...,Sienna wächst Gefühle für Darren und sie küsse...,-2.694210,-0.419699,2.274511


In [6]:
maskd_df = pd.DataFrame()
for index, text in result_dev["text_a"].items():
    try:
        wanted_index = en_de_dev_df.index[en_de_dev_df['original'] == text].tolist()[0]
        s = en_de_dev_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the dev data        
    except IndexError:
        wanted_index = None
        matched_string = process.extractOne(text,en_de_dev_df["original"].values.tolist())[0]
        wanted_index = en_de_dev_df.index[en_de_dev_df['original'] == matched_string].tolist()[0]
        wanted_row = en_de_dev_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,wanted_row], ignore_index=True)

In [7]:
maskd_df = mask(maskd_df,OUT,JSON,entities,"ende")

In [8]:
total_dev_df= pd.concat([maskd_df,en_de_dev_df], ignore_index=True)

total_dev_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

total_dev_df.to_csv("../masked-data/masked_dev_en_de_025.tsv", index = None, header=True,sep = '\t')

total_dev_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,33,NE0.,World Encyclopedia of Police Forces and Straal...,"[1, 1, 10, 1, 1, 1]",2.500000,"[-3.2428122788540117, -2.9342602894249366, -2....",-4.825809,-0.216516
1,269,In NE1.,In NE2.,"[1, 1, 1, 1, 13, 7]",4.000000,"[-3.2428122788540117, -2.9342602894249366, -3....",-3.760660,-0.167176
2,236,"When the wrestler hesitated or declined, NE3 w...","Als der Wrestler zögerte oder ablehnte, nickte...","[35, 35, 50, 15, 1, 29]",27.500000,"[-1.8104622946897808, -1.5862748751298898, -1....",-3.730812,-0.323119
3,764,The NE5 tied it in the third on a triple by NE...,Die NE6 band es in der dritten auf einem Tripl...,"[25, 1, 25, 17, 23, 28]",19.833333,"[-2.2317417017969077, -2.9342602894249366, -2....",-3.760030,-0.380381
4,802,Possibly Old South Arabian speakers dwelled fr...,MNE12erweise wohnten alte südarabische Spreche...,"[51, 51, 40, 12, 1, 22]",29.500000,"[-1.136415243318378, -0.9519287978145737, -1.5...",-3.616978,-0.671600
...,...,...,...,...,...,...,...,...
1020,995,"The gang absconded with $2,000 cash in the sec...",Die Bande flüchtete mit $2.000 Bargeld in den ...,"[70, 65, 95, 100, 77, 100]",84.500000,"[-0.33598436981483737, -0.3968759801636721, 0....",0.088897,-0.259579
1021,996,The Irish settlers arrives from Northern Irela...,Die irischen Siedler kommen kurz nach den Loya...,"[100, 95, 95, 85, 81, 100]",92.666667,"[0.9278538515065428, 0.7925229148025456, 0.746...",0.614926,-0.269823
1022,997,Volcanics include dacite breccia and small rem...,Zu den Vulkanen gehören Dacite Breccia und kle...,"[50, 50, 40, 100, 84, 92]",69.333333,"[-1.1785431840290908, -0.991575427646781, -1.5...",-0.484296,-0.412556
1023,998,The Fort Concho Museum acquired the Schoolhous...,Das Fort Concho Museum erwarb 1946 das Schulha...,"[100, 91, 95, 95, 77, 100]",93.000000,"[0.9278538515065428, 0.6339363954737166, 0.746...",0.560427,-0.208066


# Training Data

In [15]:
en_de_train_df =  pd.read_csv("../data/en-de/train.ende.df.short.tsv",sep="\t")


result_train = pd.read_csv("../masked-data/train_en_de.tsv", sep="\t")

result_train["labels"] = en_de_train_df["z_mean"]

result_train["error"] = abs(result_train["labels"]-result_train["predictions"])

In [16]:
result_train

,index,text_a,text_b,predictions,labels,error
0,0,José Ortega y Gasset visited Husserl at Freibu...,1934 besuchte José Ortega y Gasset Husserl in ...,-0.254870,1.119409,1.374278
1,1,"However, a disappointing ninth in China meant ...",Eine enttäuschende Neunte in China bedeutete j...,-0.368015,-0.488591,0.120575
2,2,"In his diary, Chase wrote that the release of ...","In seinem Tagebuch, Chase schrieb, dass die Ve...",-0.362235,-2.207007,1.844771
3,3,Heavy arquebuses mounted on wagons were called...,Schwere Arquebuses auf Waggons montiert wurden...,-0.467312,-0.799946,0.332635
4,4,Once North Pacific salmon die off after spawni...,Sobald der nordpazifische Lachs nach dem Laich...,-0.386398,0.381633,0.768031
...,...,...,...,...,...,...
6995,6995,Some may also discourage or disallow unsanitar...,Einige können auch unhygienische Praktiken wie...,-0.324950,0.544875,0.869824
6996,6996,"In the late 1860s, the crinolines disappeared ...",In den späten 1860er Jahren verschwanden die K...,-0.332789,0.271088,0.603877
6997,6997,"Disco was criticized as mindless, consumerist,...","Disco wurde als geistlos, konsumistisch, überp...",-0.334347,-1.104549,0.770203
6998,6998,Planters would then fill large hogsheads with ...,Die Pflanzer würden dann große Heuschrecken mi...,-0.361690,-0.207402,0.154289


In [17]:
result_train = result_train.sort_values(by=["error"],ascending=False)
num = int(7000 * 0.025)
result_train = result_train.head(num)

result_train.reset_index(inplace=True,drop=True)
result_train

,index,text_a,text_b,predictions,labels,error
0,6975,He vacated the WWA Cruiserweight title in Apri...,Nach seiner Rückkehr zum WWF verließ er im Apr...,-0.308271,-7.541931,7.233660
1,2080,"Haroun and S.A. Mourad, Proceedings of Interna...","Haroun and S.A. Mourad, Proceedings of Interna...",-0.361857,-7.194496,6.832639
2,6229,Boone Trails Grand Towers Gravois Trail New Ho...,Boone Trails Grand Towers Gravois Trail New Ho...,-0.500930,-7.043685,6.542755
3,2500,"""Radio Birdman"", in The Trouser Press Record G...","""Radio Birdman"", in The Hose Press Record Guid...",-0.308249,-6.847839,6.539590
4,2214,Illustrated Encyclopedia of Woodworking Handto...,Illustrated Encyclopedia of Woodworking Handto...,-0.302977,-6.832649,6.529672
...,...,...,...,...,...,...
170,2225,The Cubs' offense again struggled to score as ...,"Die Cubs 'Straftat wieder kämpfte, um Gäste, w...",-0.505674,-2.337688,1.832013
171,5772,Converted and redesignated 11 December 1951 as...,Am 11. Dezember 1951 umbenannt und umbenannt a...,-0.478009,-2.309352,1.831343
172,2612,After forcing the Patriots to punt early in th...,Nachdem Adam Jones die Patrioten gezwungen hat...,-0.445570,1.377154,1.822724
173,168,The typhoon then curved north as it rounded a ...,"Der Taifun kurvte dann nach Norden, als er ein...",-0.407987,-2.228759,1.820773


In [18]:

maskd_df = pd.DataFrame()
for index, text in result_train["text_a"].items():
    try:
        wanted_index = en_de_train_df.index[en_de_train_df['original'] == text].tolist()[0]
        s = en_de_train_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the train data        
    except IndexError:
        wanted_index = None
        matched_string = process.extractOne(text,en_de_train_df["original"].values.tolist())[0]
        wanted_index = en_de_train_df.index[en_de_train_df['original'] == matched_string].tolist()[0]
        wanted_row = en_de_train_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,wanted_row], ignore_index=True)

In [ ]:
maskd_df = mask(maskd_df,OUT,JSON,entities,"ende")

In [ ]:
total_train_df = pd.concat([maskd_df,en_de_train_df], ignore_index=True)
total_train_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

total_train_df.to_csv("../masked-data/masked_train_en_de_025.tsv", index = None, header=True,sep = '\t')

total_train_df

# same enity recogniser

In [5]:
en_de_dev_df =  pd.read_csv("../data/en-de/dev.ende.df.short.tsv",sep="\t")

result_dev = pd.read_csv("../masked-data/dev_en_de.tsv", sep="\t")


result_dev["error"] = abs(result_dev["labels"]-result_dev["predictions"])

result_dev = result_dev.sort_values(by=["error"],ascending=False)

result_dev= result_dev[result_dev["error"]>= 0.6444].reset_index(drop=True)
result_dev

,text_a,text_b,labels,predictions,error
0,World Encyclopedia of Police Forces and Penal ...,World Encyclopedia of Police Forces and Straal...,-4.825809,-0.340117,4.485691
1,In The Goldenseal Book of the West Virginia Mi...,In The Goldenseal Book of the West Virginia Mi...,-3.760660,-0.349047,3.411613
2,"When the wrestler hesitated or declined, Sting...","Als der Wrestler zögerte oder ablehnte, nickte...",-3.730812,-0.373251,3.357561
3,The Cubs tied it in the third on a triple by B...,Die Cubs band es in der dritten auf einem Trip...,-3.760030,-0.449195,3.310835
4,Possibly Old South Arabian speakers dwelled fr...,Möglicherweise wohnten alte südarabische Sprec...,-3.616978,-0.311670,3.305308
...,...,...,...,...,...
522,Femoral nerve paralysis causes hyperflexion of...,Die feminale Nervenlähmung führt zu Hyperflexi...,0.265726,-0.383941,0.649667
523,TG 52.2 then voyaged to the Gilbert Islands to...,"TG 52.2 reiste dann zu den Gilbert Inseln, um ...",0.335268,-0.313811,0.649078
524,The first Hercules landed on the new runway on...,Am 28. August landete der erste Herkules auf d...,0.376660,-0.270235,0.646895
525,Despite Thorpe's assertion that he could not m...,"Trotz Thorpes Behauptung, dass er nicht mit Sp...",0.320917,-0.325518,0.646435


In [6]:
maskd_df = pd.DataFrame()
for index, text in result_dev["text_a"].items():
    try:
        wanted_index = en_de_dev_df.index[en_de_dev_df['original'] == text].tolist()[0]
        s = en_de_dev_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the dev data        
    except IndexError:
        pass
maskd_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,33,World Encyclopedia of Police Forces and Penal ...,World Encyclopedia of Police Forces and Straal...,"[1, 1, 10, 1, 1, 1]",2.500000,"[-3.2428122788540117, -2.9342602894249366, -2....",-4.825809,-0.216516
1,269,In The Goldenseal Book of the West Virginia Mi...,In The Goldenseal Book of the West Virginia Mi...,"[1, 1, 1, 1, 13, 7]",4.000000,"[-3.2428122788540117, -2.9342602894249366, -3....",-3.760660,-0.167176
2,236,"When the wrestler hesitated or declined, Sting...","Als der Wrestler zögerte oder ablehnte, nickte...","[35, 35, 50, 15, 1, 29]",27.500000,"[-1.8104622946897808, -1.5862748751298898, -1....",-3.730812,-0.323119
3,764,The Cubs tied it in the third on a triple by B...,Die Cubs band es in der dritten auf einem Trip...,"[25, 1, 25, 17, 23, 28]",19.833333,"[-2.2317417017969077, -2.9342602894249366, -2....",-3.760030,-0.380381
4,802,Possibly Old South Arabian speakers dwelled fr...,Möglicherweise wohnten alte südarabische Sprec...,"[51, 51, 40, 12, 1, 22]",29.500000,"[-1.136415243318378, -0.9519287978145737, -1.5...",-3.616978,-0.671600
...,...,...,...,...,...,...,...,...
518,211,Femoral nerve paralysis causes hyperflexion of...,Die feminale Nervenlähmung führt zu Hyperflexi...,"[91, 95, 92, 84, 95, 71]",88.000000,"[0.5487023851101287, 0.7925229148025456, 0.620...",0.265726,-0.550166
519,574,TG 52.2 then voyaged to the Gilbert Islands to...,"TG 52.2 reiste dann zu den Gilbert Inseln, um ...","[91, 95, 92, 100, 76, 92]",91.000000,"[0.5487023851101287, 0.7925229148025456, 0.620...",0.335268,-0.381236
520,376,The first Hercules landed on the new runway on...,Am 28. August landete der erste Herkules auf d...,"[100, 95, 95, 100, 96, 73]",93.166667,"[0.9278538515065428, 0.7925229148025456, 0.746...",0.376660,-0.276205
521,278,Despite Thorpe's assertion that he could not m...,"Trotz Thorpes Behauptung, dass er nicht mit Sp...","[90, 70, 95, 92, 78, 100]",87.500000,"[0.506574444399416, -0.19864283100263583, 0.74...",0.320917,-0.371713


In [10]:
maskd_df = same_enity_mask(maskd_df,OUT,JSON,entities,"ende")

total_dev_df= pd.concat([maskd_df,en_de_dev_df], ignore_index=True)

total_dev_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

In [11]:
total_dev_df.to_csv("../masked-data/masked_dev_en_de_same.tsv", index = None, header=True,sep = '\t')

total_dev_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,33,NE0.,World Encyclopedia of Police Forces and Straal...,"[1, 1, 10, 1, 1, 1]",2.500000,"[-3.2428122788540117, -2.9342602894249366, -2....",-4.825809,-0.216516
1,269,In NE1.,In NE1.,"[1, 1, 1, 1, 13, 7]",4.000000,"[-3.2428122788540117, -2.9342602894249366, -3....",-3.760660,-0.167176
2,236,"When the wrestler hesitated or declined, NE2 w...","Als der Wrestler zögerte oder ablehnte, nickte...","[35, 35, 50, 15, 1, 29]",27.500000,"[-1.8104622946897808, -1.5862748751298898, -1....",-3.730812,-0.323119
3,764,The NE3 tied it in the third on a triple by NE...,Die NE3 band es in der dritten auf einem Tripl...,"[25, 1, 25, 17, 23, 28]",19.833333,"[-2.2317417017969077, -2.9342602894249366, -2....",-3.760030,-0.380381
4,802,Possibly Old South Arabian speakers dwelled fr...,MNE6erweise wohnten alte südarabische Sprecher...,"[51, 51, 40, 12, 1, 22]",29.500000,"[-1.136415243318378, -0.9519287978145737, -1.5...",-3.616978,-0.671600
...,...,...,...,...,...,...,...,...
1514,991,"She earned high grades, excelling in French an...","Sie verdiente hohe Noten, die sich im Französi...","[91, 80, 88, 87, 88, 88]",87.000000,"[0.5487023851101287, 0.19782346731943673, 0.45...",0.359279,-0.636871
1516,993,"Trio d'anches, or trio of reeds consists of ob...",Trio d 'anches oder Trio aus Schilf besteht au...,"[50, 50, 50, 100, 82, 72]",67.333333,"[-1.1785431840290908, -0.991575427646781, -1.1...",-0.630671,-0.198255
1518,995,"The gang absconded with $2,000 cash in the sec...",Die Bande flüchtete mit $2.000 Bargeld in den ...,"[70, 65, 95, 100, 77, 100]",84.500000,"[-0.33598436981483737, -0.3968759801636721, 0....",0.088897,-0.259579
1520,997,Volcanics include dacite breccia and small rem...,Zu den Vulkanen gehören Dacite Breccia und kle...,"[50, 50, 40, 100, 84, 92]",69.333333,"[-1.1785431840290908, -0.991575427646781, -1.5...",-0.484296,-0.412556


In [15]:
en_de_train_df =  pd.read_csv("../data/en-de/train.ende.df.short.tsv",sep="\t")


result_train = pd.read_csv("../masked-data/train_en_de.tsv", sep="\t")

result_train["labels"] = en_de_train_df["z_mean"]

result_train["error"] = abs(result_train["labels"]-result_train["predictions"])


In [16]:
result_train = result_train.sort_values(by=["error"],ascending=False)
result_train= result_train[result_train["error"]>= 0.6444].reset_index(drop=True)

In [17]:
maskd_df = pd.DataFrame()
for index, text in result_train["text_a"].items():
    try:
        wanted_index = en_de_train_df.index[en_de_train_df['original'] == text].tolist()[0]
        s = en_de_train_df.iloc[[wanted_index]]
        maskd_df = pd.concat([maskd_df,s], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the train data        
    except IndexError:
        pass
    
maskd_df =  same_enity_mask(maskd_df,OUT,JSON,entities,"ende")

In [18]:
total_train_df = pd.concat([maskd_df,en_de_train_df], ignore_index=True)
total_train_df.drop_duplicates(subset=["index"],keep="first",inplace=True)

total_train_df.to_csv("../masked-data/masked_train_en_de_same.tsv", index = None, header=True,sep = '\t')

total_train_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,6975,He vacated the NE0 title in April 2002 after r...,Nach seiner Rückkehr zum NE1 verließ er im Apr...,"[1, 13, 24]",12.666666666666666,"[-9.000885564931252, -10.29283318973243, -3.33...",-7.541931,-0.360573
1,2080,"NE2 and NE3, Proceedings of International Coll...","NE2 and NE3, Proceedings of International Coll...","[1, 1, 1]",1.0,"[-9.000885564931252, -8.00864789335575, -4.573...",-7.194496,-0.240925
2,6229,NE4 North Star Osage Ozark Trailblazers Pathfi...,NE4 North Star Osage Ozark Trailblazers Pathfi...,"[1, 1, 1]",1.0,"[-9.000885564931252, -7.556217303171075, -4.57...",-7.043685,-0.061312
3,2214,NE5.,NE5.,"[1, 1, 1]",1.0,"[-8.00864789335575, -7.556217303171075, -4.933...",-6.832649,-0.220932
4,1846,"NE6, NE7, IFWG PublishinNE8","NE6, NE7, NE8.","[1, 1, 1]",1.0,"[-8.00864789335575, -7.556217303171075, -4.933...",-6.832649,-0.129047
...,...,...,...,...,...,...,...,...
9964,6992,Those attacks exacted a toll from Beales neigh...,Diese Angriffe forderten eine Maut von Beales ...,"[100, 91, 75, 50, 39, 35]",65.0,"[0.2654514735602496, 0.04515183701325695, -0.5...",-0.611383,-0.342817
9965,6993,"Thus, even a smaller Arabian can carry a heavy...",So kann auch ein kleinerer Araber einen schwer...,"[77, 98, 88]",87.66666666666667,"[-0.32967577825642896, 0.6715584827086242, 0.4...",0.276843,-0.450241
9966,6994,"Albany fled to France, then England, where he ...","Albany floh nach Frankreich, dann England, wo ...","[100, 79, 74]",84.33333333333333,"[0.2654514735602496, -0.13950363286393827, -0....",-0.219119,-0.230298
9968,6996,"In the late 1860s, the crinolines disappeared ...",In den späten 1860er Jahren verschwanden die K...,"[100, 93, 80]",91.0,"[0.2654514735602496, 0.22412516435479046, 0.32...",0.271088,-0.317499


In [19]:
maskd_df

,index,original,translation,scores,mean,z_scores,z_mean,model_scores
0,6975,He vacated the NE0 title in April 2002 after r...,Nach seiner Rückkehr zum NE1 verließ er im Apr...,"[1, 13, 24]",12.666666666666666,"[-9.000885564931252, -10.29283318973243, -3.33...",-7.541931,-0.360573
1,2080,"NE2 and NE3, Proceedings of International Coll...","NE2 and NE3, Proceedings of International Coll...","[1, 1, 1]",1.0,"[-9.000885564931252, -8.00864789335575, -4.573...",-7.194496,-0.240925
2,6229,NE4 North Star Osage Ozark Trailblazers Pathfi...,NE4 North Star Osage Ozark Trailblazers Pathfi...,"[1, 1, 1]",1.0,"[-9.000885564931252, -7.556217303171075, -4.57...",-7.043685,-0.061312
3,2214,NE5.,NE5.,"[1, 1, 1]",1.0,"[-8.00864789335575, -7.556217303171075, -4.933...",-6.832649,-0.220932
4,1846,"NE6, NE7, IFWG PublishinNE8","NE6, NE7, NE8.","[1, 1, 1]",1.0,"[-8.00864789335575, -7.556217303171075, -4.933...",-6.832649,-0.129047
...,...,...,...,...,...,...,...,...
2967,6961,Sweden and Finland also augmented their roster...,Schweden und Finnland erweitert auch ihre Dien...,"[83, 98, 97]",92.66666666666667,"[0.7990540222688698, 0.20584462994058395, -0.0...",0.329851,-0.315033
2968,4410,East Pakistan objected and a protracted disput...,"Ostpakistan lehnte ab, und es kam zu einem lan...","[100, 94, 97]",97.0,"[0.2654514735602496, 0.3136118280255572, 0.504...",0.361346,-0.473948
2969,2247,The second group of historians present a class...,Die zweite Gruppe von Historikern präsentiert ...,"[100, 87, 77]",88.0,"[0.2654514735602496, 0.6211849487060245, -0.15...",0.244986,-0.289096
2970,3207,The central pairing of Maniche and NE3250 targ...,Die zentrale Paarung von Maniche und NE3250 zi...,"[100, 88, 94]",94.0,"[0.2654514735602496, -0.2233081539990433, 0.62...",0.222210,-0.256297
